This is a simple tutorial to map a model to the AKIDA1000 hardware

In [1]:
import tensorflow as tf
from tensorflow import keras
from quantizeml.models import quantize
from cnn2snn import convert
import os
import datetime
import tensorflow as tf
from tensorflow import keras
import cnn2snn
import akida as ak
from quantizeml.models import quantize
from quantizeml.layers import QuantizationParams


2025-11-08 18:50:28.576558: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-08 18:50:28.632174: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-08 18:50:28.632208: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-08 18:50:28.633869: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-08 18:50:28.642932: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-08 18:50:28.643483: I tensorflow/core/platform/cpu_feature_guard.cc:1

Setting the akida env version to v1 is very important to port it to hardware

In [2]:
os.environ["CNN2SNN_TARGET_AKIDA_VERSION"] = "v1"
print('----------------------------------')
print("    TF version: ", tf.__version__)
print("MetaTF version: ", ak.__version__)
print(' Akida version: ', cnn2snn.get_akida_version()) 
print('----------------------------------')

!which python

today = datetime.datetime.now()
todayAsStr = str(today.year) + '-' + str(today.month) + '-' + str(today.day) + '_' + str(today.hour) + '-' + str(today.minute)
print('Time of run: ', todayAsStr)

----------------------------------
    TF version:  2.15.0
MetaTF version:  2.11.0
 Akida version:  AkidaVersion.v1
----------------------------------
/home/kannan/anaconda3/envs/tf_env/bin/python
Time of run:  2025-11-8_18-50


In [3]:
# 1. Load trained model 
model_src = keras.models.load_model("ESA_cnn.h5", compile=False)
model_src.summary()


Model: "akida_friendly_cnn_v5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 60, 76, 1)]       0         
                                                                 
 conv1 (Conv2D)              (None, 60, 76, 16)        416       
                                                                 
 conv1_relu (ReLU)           (None, 60, 76, 16)        0         
                                                                 
 pool1 (MaxPooling2D)        (None, 30, 38, 16)        0         
                                                                 
 conv2 (Conv2D)              (None, 30, 38, 32)        4640      
                                                                 
 conv2_relu (ReLU)           (None, 30, 38, 32)        0         
                                                                 
 pool2 (MaxPooling2D)        (None, 15, 19, 3

In [4]:
compatibilityIssues = cnn2snn.check_model_compatibility(model_src, device=ak.AKD1000())
print('Compatibility issues: ', compatibilityIssues)


1/1 [==============================] - 0s 340ms/step


/home/kannan/anaconda3/envs/tf_env/lib/python3.10/site-packages/cnn2snn/quantizeml/blocks.py:160: UserWarning: Conversion stops at layer logit because of a dequantizer. The end of the model is ignored:
___________________________________________________
Layer (type)
output (Activation)

  warnings.warn("Conversion stops" + stop_layer_msg + " because of a dequantizer. "


Compatibility issues:  None


In [ ]:
from quantizeml.models import quantize
from quantizeml.layers import QuantizationParams

# Quantize the model
qparams = QuantizationParams(input_weight_bits=8, weight_bits=4, activation_bits=4, per_tensor_activations=True)
model_quantized = quantize(model_src, qparams=qparams)

In [ ]:
print(' Akida version check: ', cnn2snn.get_akida_version()) 
akida_model = cnn2snn.convert(model_quantized, file_path='akida_model.fbz')

In [ ]:
akida_model = cnn2snn.convert(model_quantized, file_path='akida_model.fbz')

In [ ]:
print('Model version: ', akida_model.ip_version, '\n')